<a href="https://colab.research.google.com/github/geocarvalho/uni-proj/blob/master/IN1164/project_1/PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementação: 
- PSO padrão e uma variação.
- ABC (ou Firefly) padrão e uma variação.


Testar em 6 funções de benchmark com variação de atributos de 10, 20 e 50.
Funções:
- [Ackley function](http://benchmarkfcns.xyz/benchmarkfcns/ackleyfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/ackley.html)
- [Alpine function](http://benchmarkfcns.xyz/benchmarkfcns/alpinen1fcn.html)
- [Schwefel Function](http://benchmarkfcns.xyz/benchmarkfcns/schwefelfcn.html) - [em python](https://www.cs.unm.edu/~neal.holts/dga/benchmarkFunction/schwefel.html)
- [Happy Cat Function](http://benchmarkfcns.xyz/benchmarkfcns/happycatfcn.html) 
- [Brown function](http://benchmarkfcns.xyz/benchmarkfcns/brownfcn.html)
- [Exponential function](http://benchmarkfcns.xyz/benchmarkfcns/exponentialfcn.html)

Para funções olhar o site: http://benchmarkfcns.xyz/

# Funções de fitness


In [10]:
import math
import random
import numpy as np

def simple_func(position):
  """ Simple function that models the problem """
  return position[0]**2 + position[1]**2 + 1

def sphere_func(positions):
  """ Sphere function 
  http://benchmarkfcns.xyz/benchmarkfcns/spherefcn.html"""
  c = np.array(positions)
  x = c**2
  return np.sum(x)

def ackley_func(positions):
  """ Ackley function"""
  c = np.array(positions)
  firstSum = np.sum(c**2.0)
  secondSum = np.sum(np.cos(2.0*math.pi*c))
  n = float(len(positions))
  return -20.0*math.exp(-0.2*math.sqrt(firstSum/n)) - math.exp(secondSum/n) + 20 + math.e

def alpine_func(positions):
  """ Alpine function """
  c = np.array(positions)
  scores = np.sum(abs(c * np.sin(c) + 0.1*c))
  return scores

def schwefel_func(positions):
  """F7 Schwefel's function
  multimodal, asymmetric, separable"""
  c = np.array(positions)
  n = len(c)
  alpha = 418.982887
  fitness = np.sum(c * np.sin(np.sqrt(abs(c))))
  return alpha * n - fitness

def happy_cat_func(positions):
  """ Happy Cat function """
  alpha = 0.5
  c = np.array(positions)
  n = len(c)
  x2 = np.sum(c*c)
  scores = (((x2 - n)**2)**alpha + (0.5*x2 + np.sum(c)))/ (n + 0.5)
  return scores

def brown_func(positions):
  """ Brown function """
  c = np.array(positions)
  n = len(c)
  x = c**2
  scores = 0
  for i in range(n-1):
    scores = scores + x[i]**(x[i+1] + 1) + x[i+1]**(x[i]+1)
  return scores

def exponential_func(positions):
  """ Exponential function """
  c = np.array(positions)
  x2 = c**2
  scores = -np.exp(-0.5 * np.sum(x2))
  return scores


# PSO padrão

* [1995 Particle swarm optimization](https://ieeexplore.ieee.org/abstract/document/488968)
* [1998 A Modified Particle Swarm Optimizer](https://ieeexplore.ieee.org/document/699146)

In [8]:
def PSO(problem, dimension, var_min, var_max, n_iterations, n_particles,
        wdamp, w, c1, c2, show_iter):
  """ PSO algorithm """
  # Inicialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = particle_position_vector
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.array([float('inf'), float('inf')])
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0

  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = particle_position_vector[p]
      
      # Calculate gbest
      if gbest_fitness_value > fitness_candidate:
        gbest_fitness_value = fitness_candidate
        gbest_position = particle_position_vector[p]
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      particle_position_vector[p] = new_position

    iteration += 1
    w *= wdamp

  # print("The best position is: ", gbest_position, " with value of ", gbest_fitness_value,
  #       " in iteration number ", iteration)
  return gbest_position, gbest_fitness_value

In [9]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 1000, "n_particles": 50,
              "wdamp": 0.99, "w": 1, "c1": 2, "c2": 2, "show_iter": False}
    gbest_pos, gbest_value = PSO(**kwargs)
    print("For dimension %s using the %s, the gbest was: %s" % (dim, name, gbest_value))

For dimension 10 using the Ackley function, the gbest was: 4.836432354192709
For dimension 10 using the Alpine function, the gbest was: 4.762118976567248
For dimension 10 using the Schwefel's function, the gbest was: 4153.872640208571
For dimension 10 using the Happy Cat function, the gbest was: 0.9507230529595402
For dimension 10 using the Brown function, the gbest was: 6.795120099016241
For dimension 10 using the Exponential function, the gbest was: -0.24433214712703163
For dimension 20 using the Ackley function, the gbest was: 7.585733200554342
For dimension 20 using the Alpine function, the gbest was: 17.372731716325294
For dimension 20 using the Schwefel's function, the gbest was: 8330.831575163122
For dimension 20 using the Happy Cat function, the gbest was: 1.0969676118090352
For dimension 20 using the Brown function, the gbest was: 408.26967787457176
For dimension 20 using the Exponential function, the gbest was: -3.523722372723226e-08
For dimension 50 using the Ackley function

# Variação do PSO

* [2002 The particle swarm - explosion, stability, and convergence in a multidimensional complex space](https://ieeexplore.ieee.org/abstract/document/985692)

> Constriction coefficients

$\chi = \frac{2k}{|2 - \phi - \sqrt{\phi^2 - 4\phi}|}$

* Onde $0\leq k \leq 1$ e $\phi_1, \phi_2 = 2.05$;
* Valor de $k=1$;
* Substituições $w = \chi$, $c_1 = \chi.\phi_1$ e $c_2 = \chi.\phi_2$.
* Além disso, $\phi = \phi_1 + \phi_2 \geq 4$

In [15]:
def PSO_cc(problem, dimension, var_min, var_max, n_iterations, n_particles,
        wdamp, w, c1, c2, show_iter):
  """ PSO algorithm """
  # Inicialization
  # np.seterr(over='ignore')
  particle_position_vector = np.random.uniform(var_min,var_max,(
      n_particles, dimension))
  pbest_position = particle_position_vector
  pbest_fitness_value = np.full(shape=n_particles, fill_value=float('inf'))
  gbest_fitness_value = float('inf')
  gbest_position = np.array([float('inf'), float('inf')])
  velocity_vector = np.zeros(shape=(n_particles, dimension))
  iteration = 0

  # Start iterations
  while iteration < n_iterations:
    for p in range(n_particles):
      fitness_candidate = problem(particle_position_vector[p])
      
      # Calculate pbest
      if pbest_fitness_value[p] > fitness_candidate:
        pbest_fitness_value[p] = fitness_candidate
        pbest_position[p] = particle_position_vector[p]
      
      # Calculate gbest
      if gbest_fitness_value > fitness_candidate:
        gbest_fitness_value = fitness_candidate
        gbest_position = particle_position_vector[p]
    if show_iter:
      print(gbest_fitness_value, " :", gbest_position)
    
    # Update velocity of each particle
    for p in range(n_particles):
      new_velocity = (w * velocity_vector[p]) + \
      ((c1 * random.random()) * (pbest_position[p] - particle_position_vector[p])) + \
      ((c2 * random.random()) * (gbest_position - particle_position_vector[p]))
      new_position = new_velocity + particle_position_vector[p]
      particle_position_vector[p] = new_position

    iteration += 1
    w *= wdamp

  # print("The best position is: ", gbest_position, " with value of ", gbest_fitness_value,
  #       " in iteration number ", iteration)
  return gbest_position, gbest_fitness_value

In [16]:
# Problem definition
fitness_functions = {"Ackley function": ackley_func, 
                     "Alpine function": alpine_func, 
                     "Schwefel's function": schwefel_func, 
                     "Happy Cat function": happy_cat_func,
                     "Brown function": brown_func,
                     "Exponential function": exponential_func}
dimensions = [10, 20, 50]

phi1 = 2.05
phi2 = 2.05
phi = phi1 + phi2
kappa = 1
chi = 2*kappa/(abs(2-phi-math.sqrt(phi**2 - 4*phi)))

for dim in dimensions:
  for name, func in fitness_functions.items():
    kwargs = {"problem": func, "dimension": dim, "var_min": -10, 
              "var_max": 10, "n_iterations": 1000, "n_particles": 50,
              "wdamp": 1, "w": chi, "c1": chi*phi1, "c2": chi*phi2, "show_iter": False}
    gbest_pos, gbest_value = PSO_cc(**kwargs)
    print("For dimension %s using the %s, the gbest was: %s" % (dim, name, gbest_value))

For dimension 10 using the Ackley function, the gbest was: 5.482614393629319
For dimension 10 using the Alpine function, the gbest was: 5.284884444645204
For dimension 10 using the Schwefel's function, the gbest was: 4162.266288272899
For dimension 10 using the Happy Cat function, the gbest was: 1.2984748809579425
For dimension 10 using the Brown function, the gbest was: 21.2898701014414
For dimension 10 using the Exponential function, the gbest was: -8.492091376944449e-06
For dimension 20 using the Ackley function, the gbest was: 7.438340349306808
For dimension 20 using the Alpine function, the gbest was: 19.522356352826066
For dimension 20 using the Schwefel's function, the gbest was: 8318.120414521023
For dimension 20 using the Happy Cat function, the gbest was: 1.7546537364132178
For dimension 20 using the Brown function, the gbest was: 945.6573796677054
For dimension 20 using the Exponential function, the gbest was: -1.1184914514302178e-09
For dimension 50 using the Ackley functio

# Outras referências

* [A novel particle swarm optimization algorithm with adaptive inertia weight](https://www.sciencedirect.com/science/article/abs/pii/S156849461100055X)
* [The particle swarm optimization algorithm: convergence analysis and parameter selection](https://www.sciencedirect.com/science/article/abs/pii/S0020019002004477)
* [An Overview of Particle Swarm Optimization Variants](https://www.sciencedirect.com/science/article/pii/S1877705813001823)
* [2020 Population size in Particle Swarm Optimization](https://www.sciencedirect.com/science/article/pii/S2210650220303710)

In [12]:
phi1 = 2.05
phi2 = 2.05
phi = phi1 + phi2
kappa = 1
chi = 2*kappa/(abs(2-phi-math.sqrt(phi**2 - 4*phi)))
chi

0.7298437881283576